In [ ]:
!pip install mtcnn
!pip install --upgrade youtube_dl
from google.colab import files
import ipywidgets as widgets
import os
import cv2
import matplotlib.pyplot as plt
from mtcnn import MTCNN
import numpy as np
from scipy.spatial import distance
from google.colab.patches import cv2_imshow
!pip install youtube_dl
from scipy.spatial import distance


In [ ]:
# UI
# 옵션 선택 핸들러 함수
def handle_option_selection(change):
    selected_option = change['new']
    print(f'선택된 옵션: {selected_option}')


# 비디오 파일 업로드 핸들러 함수
def handle_video_upload(change):
    uploaded_video = list(change['new'].values())[0]
    
    # 비디오 파일을 videos 폴더에 저장
    file_path = os.path.join('/content/videos', uploaded_video['metadata']['name'])
    with open(file_path, 'wb') as f:
        f.write(uploaded_video['content'])
    
    print(f'비디오 저장 완료: {file_path}')

# 이미지 파일 업로드 핸들러 함수
def handle_image_upload(change):
    uploaded_images = change['new']
    
    for uploaded_image in uploaded_images.values():
        # 각 이미지 파일을 info 폴더에 저장
        file_path = os.path.join('/content/info', uploaded_image['metadata']['name'])
        with open(file_path, 'wb') as f:
            f.write(uploaded_image['content'])
    
        print(f'이미지 저장 완료: {file_path}')
        
        # 업로드한 이미지 표시
        image_widget = widgets.Image(value=uploaded_image['content'], format='jpg', width=300, height=300)
        display(image_widget)

def upload():
    # videos 폴더가 없으면 생성
    if not os.path.exists('/content/videos'):
        os.makedirs('/content/videos')

    # info 폴더가 없으면 생성
    if not os.path.exists('/content/info'):
        os.makedirs('/content/info')
  
    # 비디오 파일 업로드 위젯 생성
    video_upload = widgets.FileUpload(description='비디오 업로드')

    # 이미지 파일 업로드 위젯 생성
    image_upload = widgets.FileUpload(description='이미지 업로드', multiple=True)

    # 파일 업로드 핸들러 연결
    video_upload.observe(handle_video_upload, names='value')
    image_upload.observe(handle_image_upload, names='value')

    # 옵션 버튼
    options = ['short sleeve top', 'long sleeve top', 'short sleeve outwear', 'long sleeve outwear', 'vest', 'sling', 'shorts', 'trousers', 'skirt', 'short sleeve dress', 'long sleeve dress', 'vest dress', 'sling dress']
    option_button = widgets.Dropdown(options=options, description='clothes')

    # 옵션 선택 핸들러 연결
    option_button.observe(handle_option_selection, names='value')

    # 위젯 표시
    display(video_upload)
    display(image_upload)
    display(option_button)


In [ ]:
#face detectection & recognition
#이미지 폴더 경로를 받아서 info_file 경로에 해당얼굴정보를 파일이름과 함께 저장하는 함수
def detect_faces_and_save_info(folder_path, info_file):
    # MTCNN 모델 초기화
    mtcnn = MTCNN()

    # 폴더 내의 이미지 파일들에 대해 처리
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            faces = mtcnn.detect_faces(image)

            # 이미지 파일명과 얼굴 정보를 파일에 저장
            with open(info_file, 'a') as f:
                for face_idx, face in enumerate(faces):
                    x, y, width, height = face['box']
                    keypoints = face['keypoints']
                    left_eye = keypoints['left_eye']
                    right_eye = keypoints['right_eye']
                    nose = keypoints['nose']
                    mouth_left = keypoints['mouth_left']
                    mouth_right = keypoints['mouth_right']

                    f.write(f"Image: {filename}\n")
                    f.write(f"Face {face_idx+1}:\n")
                    f.write(f" - Location: x={x}, y={y}, width={width}, height={height}\n")
                    f.write(f" - Landmarks:\n")
                    f.write(f"   - Left Eye: {left_eye}\n")
                    f.write(f"   - Right Eye: {right_eye}\n")
                    f.write(f"   - Nose: {nose}\n")
                    f.write(f"   - Left Mouth: {mouth_left}\n")
                    f.write(f"   - Right Mouth: {mouth_right}\n")
                    f.write("\n")

def find_similar_face_info(video_folder_path, info_file, similarity_threshold):
    # MTCNN 모델 초기화
    mtcnn = MTCNN()

    # 얼굴 정보를 담을 리스트 초기화
    face_info_list = []

    # 얼굴 정보 파일에서 얼굴 정보 읽어오기
    with open(info_file, 'r') as f:
        file_content = f.read()
        face_info_list = file_content.split('\n\n')
    # codec설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # 비디오 load
    video_files = os.listdir(video_folder_path)
    
    # 폴더 내 video 파일들 처리
    for video_file in video_files:
        video_path = os.path.join(video_folder_path, video_file)
        cap = cv2.VideoCapture(video_path)

        # 비디오 파일 정상적으로 존재하는지 확인
        if not cap.isOpened():
            print(f'Failed to open the video file: {video_path}')
            continue

        # 비디오 정보 get
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        #processed_비디오이름.mp4 파일로 후처리 영상 저장
        # Create the VideoWriter object to save the output video
        
        output_video_path = os.path.join(video_folder_path, f"processed_{video_file}")
        output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        # 프레임 마다 얼굴 검출 및 인식 후 모자이크 처리
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            # faces 에 얼굴을 찾아서 저장
            faces = mtcnn.detect_faces(frame)

            
            for face in faces:
                x, y, width, height = face['box']
                keypoints = face['keypoints']
                left_eye = keypoints['left_eye']
                right_eye = keypoints['right_eye']
                nose = keypoints['nose']
                mouth_left = keypoints['mouth_left']
                mouth_right = keypoints['mouth_right']

                target_landmarks = [left_eye, right_eye, nose, mouth_left, mouth_right]
                target_landmarks_flat = [coord for landmark in target_landmarks for coord in landmark]

                # 유사한 얼굴 있는지 확인
                similar_filename = ""
                similarity = float('inf')
                match_found = False

                for face_info in face_info_list:
                    face_lines = face_info.split('\n')
                    if len(face_lines) < 5:
                        continue

                    info_filename = face_lines[0].split(': ')[1]
                    info_landmarks = []
                    for line in face_lines[4:]:
                        landmark = line.split(': ')[1]
                        info_landmarks.append(list(map(int, landmark.strip('()').split(', '))))


                    info_landmarks_flat = [coord for landmark in info_landmarks for coord in landmark]

                    # 랜드마크 사이의 거리비를 바탕으로 유사도 측정
                    distance_metric = distance.euclidean(target_landmarks_flat, info_landmarks_flat)

                    # info폴더내 얼굴중 가장 유사한 얼굴찾기
                    if distance_metric < similarity:
                        similarity = distance_metric
                        similar_filename = info_filename
                        match_found = True

                # threshold보다 유사도가 낮으면 누구인지 표시 / 높으면 모자이크 처리
                if match_found and similarity <= similarity_threshold:
                    # 유사도와 인물이름 표시
                    text = f"{similar_filename} ({similarity:.2f})"
                    cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
                    cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
                else:
                    # 모자이크 처리
                    text = f"Mosaic ({similarity:.2f})"
                    frame[y:y+height, x:x+width] = cv2.blur(frame[y:y+height, x:x+width], (30, 30))
                    cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2, cv2.LINE_AA)

            # 후처리 frame 저장
            output_video.write(frame)

            # 중간점검용 frame imshow
            cv2_imshow( frame)

            

        #
        cap.release()
        output_video.release()

  


In [ ]:
#업로드 ui실행  및 파일 업로
upload()

In [ ]:
#메인문


# 경로설정
info_folder_path = '/content/info'  # 방송인 얼굴 지정 폴더
info_file_path = '/content/info.txt'  # 위 경로에 방송인 얼굴정보 저장됨. 메인문 실행하면 위치에 자동 생성됨
video_folder_path = '/content/videos'  # 처리할 영상의 폴더 경로
similarity_threshold = 1300 # threshold 설정 낮을 수록 유사함 

# info_folder_path 경로를 주면 info_file_path 경로에 얼굴정보를 저장
detect_faces_and_save_info(info_folder_path, info_file_path)
#비디오 경로와 detect_faces 함수로 생성한 info_file을 주면 해당 비디오에서 threshold 이상의 얼굴을 모자이크 처리함
find_similar_face_info(video_folder_path, info_file_path, similarity_threshold)
